## 2. 데이터셋 구성하기


In [1]:
import torch
from torch.utils.data import Dataset
import json
import os
from PIL import Image
from materials.utils import *
import warnings
warnings.filterwarnings('ignore')

class WeaponDataset(Dataset):
    """
    A PyTorch Dataset class to be used in a PyTorch DataLoader to create batches.
    """

    def __init__(self, data_folder, keep_difficult=False):
        """
        :param data_folder: folder where data files are stored
        :param split: split, one of 'TRAIN' or 'TEST'
        :param keep_difficult: keep or discard objects that are considered difficult to detect?
        """
        self.keep_difficult = keep_difficult
        self.data_folder = data_folder
        self.keep_difficult = keep_difficult

        # Read data files
        with open(os.path.join(data_folder, 'TRAIN' + '_images.json'), 'r') as j:
            self.images = json.load(j)
        with open(os.path.join(data_folder, 'TRAIN' + '_objects.json'), 'r') as j:
            self.objects = json.load(j)

        assert len(self.images) == len(self.objects)

    def __getitem__(self, i):
        # Read image
        image = Image.open(self.images[i], mode='r')
        image = image.convert('RGB')

        # Read objects in this image (bounding boxes, labels, difficulties)
        objects = self.objects[i]
        boxes = torch.FloatTensor(objects['boxes'])  # (n_objects, 4)
        labels = torch.LongTensor(objects['labels'])  # (n_objects)
        difficulties = torch.ByteTensor(objects['difficulties'])  # (n_objects)

        # Discard difficult objects, if desired
        if not self.keep_difficult:
            boxes = boxes[1 - difficulties]
            labels = labels[1 - difficulties]
            difficulties = difficulties[1 - difficulties]

        # Apply transformations
        image, boxes, labels, difficulties = temp_transform(image, boxes, labels, difficulties, split='TRAIN')

        return image, boxes, labels, difficulties

    def __len__(self):
        return len(self.images)

    def collate_fn(self, batch):
        """
        Since each image may have a different number of objects, we need a collate function (to be passed to the DataLoader).

        This describes how to combine these tensors of different sizes. We use lists.

        Note: this need not be defined in this Class, can be standalone.

        :param batch: an iterable of N sets from __getitem__()
        :return: a tensor of images, lists of varying-size tensors of bounding boxes, labels, and difficulties
        """

        images = list()
        boxes = list()
        labels = list()
        difficulties = list()

        for b in batch:
            images.append(b[0])
            boxes.append(b[1])
            labels.append(b[2])
            difficulties.append(b[3])

        images = torch.stack(images, dim=0)

        return images, boxes, labels, difficulties  # tensor (N, 3, 300, 300), 3 lists of N tensors each

In [2]:
train_dataset = WeaponDataset('./data/Weapon')
img, boxes, labels, difficulties = train_dataset.__getitem__(0)

print(img.shape)
print(boxes)
print(labels)
print(difficulties)

FileNotFoundError: [Errno 2] No such file or directory: '/home/gihun/jupyter/LG_project/Lecture/data/Weapon/TrainData/PUBGGunsInRealLife!06681.jpg'

In [3]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(dataset=train_dataset, batch_size=16, 
                              shuffle=True, num_workers=4,
                              collate_fn=train_dataset.collate_fn)

In [4]:
img, boxes, labels, _ = next(iter(train_dataloader))